In [1]:
pip install tweepy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import tweepy
import ssl
import pandas as pd
import io
import time
from datetime import datetime

## Create Connection

In [2]:
token = 'insert your token'
client = tweepy.Client(bearer_token=token, wait_on_rate_limit=True)
apikey = "insert your apikey"
apisecretkey = "insert your api secret key"
accesstoken = "insert your access token"
accesstokensecret = "insert your secret access token"
auth = tweepy.OAuthHandler(apikey,apisecretkey) 
auth.set_access_token(accesstoken,accesstokensecret) 
api = tweepy.API(auth,wait_on_rate_limit=True)

In [3]:
# Save senators usernames here --> due to api limitations we cannot download the twees of all senators together
# Experience showed that 2-3 senators at a time work
usernames = ["BillRedpath", "ConnorVlakancic"]


Pull data
.items(n) returns n tweets => 100 items result in 1 request.

For api.search_30_day we have a limit of 250 requests and a tweet cap of 25k tweets - This is a PREMIUM SEARCH API that allows us to query beyond 7 days.

There is an api.search_full_archive that allows us to query till 15 years back but it has even tighter limits.

If you want to pull replies by tweets - MAKE SURE YOU'RE NOT EXHAUSTING YOUR LIMITS - One alternative is to try APIv2 search endpoint but it only returns tweets and replies upto last 7 days.

Note: you cannot access Premium APIs without elevated access.

Read more on Twitter APIs and Tweepy Documentation.

https://developer.twitter.com/en/docs/twitter-api/premium

https://developer.twitter.com/en/docs/twitter-api/v1



https://docs.tweepy.org/en/stable/api.html - For Twitter API v1

https://docs.tweepy.org/en/stable/client.html - For Twitter API v2




---



## Initiate Twitter connection and get relevant data (users, tweets, replies)

In [ ]:
userList = []
tweetList = []
replyList = []
for username in usernames:
  # get user profile from username
  user = client.get_user(username=username, user_fields=['public_metrics', 'created_at', 'description', 'location', 'verified'])
  userList.append({
      "name":user[0]["name"],
      "username":username,
      "id":user[0]["id"],
      "public_metrics": user[0]["public_metrics"],
      "description": user[0]["description"],
      "location": user[0]["location"],
      "verified": user[0]["verified"]
  })
  tweetIDs = set()

  # get tweets by user
  time.sleep(0.01)
  tweets = tweepy.Cursor(api.search_30_day, label="dev", query = f"from:{username} lang:en", fromDate="202201010000").items(200)
  for tweet in tweets:    
    tweet = tweet._json
    hashtags = [tweet["entities"]["hashtags"][i]["text"] for i in range(len(tweet["entities"]["hashtags"]))] 
    if "extended_tweet" in tweet:
      tweet["text"] = tweet["extended_tweet"]["full_text"]
      hashtags = [tweet["extended_tweet"]["entities"]["hashtags"][i]["text"] for i in range(len(tweet["extended_tweet"]["entities"]["hashtags"]))] 
    tweetList.append({
        "tweetId": tweet["id"],
        "text": tweet["text"],
        "username": username,
        "quotes":tweet["quote_count"],
        "favorites": tweet["favorite_count"],
        "replies": tweet['reply_count'],
        "retweets": tweet["retweet_count"],
        "created_at": tweet["created_at"],
        "hashtags":  hashtags      

    })
    tweetIDs.add(tweet["id"])

  # get latest replies for a user - cannot filter on conversation ID because each tweet ID will result in a single request and we are allowed only 250 requests 
  replies = tweepy.Cursor(api.search_30_day, label="dev", query = f"to:{username} lang:en", fromDate="202209150000").items(1000)
  for reply in replies:
    reply = reply._json
    reply_to = reply["in_reply_to_status_id"]
    if reply_to not in tweetIDs:
      continue
    hashtags = [reply["entities"]["hashtags"][i]["text"] for i in range(len(reply["entities"]["hashtags"]))]
    if "extended_tweet" in reply:
      reply["text"] = reply["extended_tweet"]["full_text"]
      hashtags = [reply["extended_tweet"]["entities"]["hashtags"][i]["text"] for i in range(len(reply["extended_tweet"]["entities"]["hashtags"]))]
    replyList.append({
        "tweetId": reply["id"],
        "replyTo": reply_to,
        "text": reply["text"],
        "created_at": reply["created_at"],
        "quotes":reply["quote_count"],
        "favorites": reply["favorite_count"],
        "replies": reply['reply_count'],
        "retweets": reply["retweet_count"],
        "hashtags": hashtags,
        "userId": reply["user"]["id"],
        "username":reply["user"]["screen_name"],
        "name": reply["user"]["name"],
        "followers":reply["user"]["followers_count"],
        "following":reply["user"]["friends_count"],
        "verified":reply["user"]["verified"]



        
    })

In [ ]:
## Create dataframes of the downloaded data and export them to excel files

df1 = pd.DataFrame(userList)
df2 = pd.DataFrame(tweetList)
df3 = pd.DataFrame(replyList)

In [ ]:
df1.to_excel("users.xlsx", index=False)
df2.to_excel("tweets.xlsx", index=False)
df3.to_excel("reply.xlsx", index=False)

In [ ]:
files.download("users.xlsx")
files.download("tweets.xlsx")
files.download("reply.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>